# Convert recorded Json file data to Sqlite
Still a bit trail and error, but the Json format is fixed enough to start working on full file conversion and upload into the Sqlite database.

In [1]:
from datetime import datetime
import json
import sqlite3
import pandas as pd

In [2]:
def getKnownLeaders(cnx):
    cur = cnx.cursor()
    sqlSelect = 'SELECT leaderId, name, civ FROM leader'
    return pd.read_sql_query(sqlSelect, cnx)

def getLeaderId(name, civ, knownLeaders):
    try:
        # The iloc is neccessary as we can't assume there is only one return value!?
        # also have to make sure it is int as numpy.int64 doesn't go into sqlite so well?
        return int(knownLeaders[(knownLeaders.name == name) & (knownLeaders.civ == civ)].iloc[0].leaderId)
    except IndexError:
        return None

In [3]:
jsonData = open('RawData/20181126_01.json', encoding='utf8').read()

In [4]:
rawData = json.loads(jsonData)

In [5]:
cnx = sqlite3.connect('Database/Civ6CitySettledData.db')
cur = cnx.cursor()
print(cnx)
print(cur)

## Populate the game table

In [6]:
# Python creates a list by default, we know we only have 1 so select first element only!
gameData = rawData['gameConfig'][0]
print(gameData)
print(type(gameData))

{'leader': 'Peter', 'players': [{'playerCiv': 'Russia', 'playerName': 'Peter'}, {'playerCiv': 'America', 'playerName': 'Teddy Roosevelt'}, {'playerCiv': 'Greece', 'playerName': 'Pericles'}, {'playerCiv': 'Kongo', 'playerName': 'Mvemba a Nzinga'}, {'playerCiv': 'Zulu', 'playerName': 'Shaka'}, {'playerCiv': 'Mongolia', 'playerName': 'Genghis Khan'}, {'playerCiv': 'Cree', 'playerName': 'Poundmaker'}, {'playerCiv': 'Mapuche', 'playerName': 'Lautaro'}, {'playerCiv': 'Brussels', 'playerName': 'Brussels'}, {'playerCiv': 'Nan Madol', 'playerName': 'Nan Madol'}, {'playerCiv': 'Antioch', 'playerName': 'Antioch'}, {'playerCiv': 'Muscat', 'playerName': 'Muscat'}, {'playerCiv': 'Stockholm', 'playerName': 'Stockholm'}, {'playerCiv': 'Vilnius', 'playerName': 'Vilnius'}, {'playerCiv': 'Granada', 'playerName': 'Granada'}, {'playerCiv': 'Valletta', 'playerName': 'Valletta'}, {'playerCiv': 'Jerusalem', 'playerName': 'Jerusalem'}, {'playerCiv': 'Buenos Aires', 'playerName': 'Buenos Aires'}, {'playerCiv': 

In [9]:
dateString = rawData['date']
date = datetime.strptime(dateString, '%Y%m%d').date()
game = {
    'gameId': None,
    'date': date,
    'difficulty': gameData['difficulty'],
    'mapType': 'Continents',
    'mapSize': gameData['mapSize'],
    'gameSpeed': gameData['gameSpeed'],
    'startEra': gameData['startEra'],
    'ruleSet': gameData['ruleSet'],
}
print(game)

{'gameId': None, 'date': datetime.date(2018, 11, 26), 'difficulty': 'Prince', 'mapType': 'Continents', 'mapSize': 'Standard', 'gameSpeed': 'Standard', 'startEra': 'Ancient Era', 'ruleSet': 'RULESET_EXPANSION_1'}


In [10]:
cur.execute('INSERT INTO game VALUES(:gameId, :date, :difficulty, :mapType, :mapSize, :gameSpeed, :startEra, :ruleSet)',
    game)
gameId = cur.lastrowid
print("gameID: ", gameId)

gameID:  4


In [11]:
cnx.commit()

## Add "new" leaders to the leader table.
The leader table is a list of all the leaders we've captured data for, regardless of when they were first seen. We record the relationship between a specific game/instance with the leaders in the gameLeader table.

In [12]:
knownLeaders = getKnownLeaders(cnx)
print(knownLeaders)

    leaderId                 name            civ
0          7            Amanitore          Nubia
1         35              Antioch        Antioch
2         14               Armagh         Armagh
3         28             Auckland       Auckland
4          4              Babylon        Babylon
5          3        Bandar Brunei  Bandar Brunei
6         13             Brussels       Brussels
7         11  Catherine de Medici         France
8         22            Cleopatra          Egypt
9         26                Cyrus         Persia
10        12               Geneva         Geneva
11        21         Genghis Khan       Mongolia
12         1            Gilgamesh        Sumeria
13        20              Gitarja      Indonesia
14        24      Harald Hardrada         Norway
15        36              Hattusa        Hattusa
16         5            Hong Kong      Hong Kong
17         6              Jadwiga         Poland
18        30            Jerusalem      Jerusalem
19        16        

In [15]:
result = []
for leader in gameData['players']:
    name = leader['playerName']
    civ = leader['playerCiv']
    leaderId = getLeaderId(name, civ, knownLeaders)
    if leaderId is None:
        newLeader = {
            'leaderId': None,
            'name': name,
            'civ': civ,
            'isMajor': False if (name == civ) else True
        }
        result.append(newLeader)

for row in result:
    print(row)

{'leaderId': None, 'name': 'Teddy Roosevelt', 'civ': 'America', 'isMajor': True}
{'leaderId': None, 'name': 'Pericles', 'civ': 'Greece', 'isMajor': True}
{'leaderId': None, 'name': 'Mvemba a Nzinga', 'civ': 'Kongo', 'isMajor': True}
{'leaderId': None, 'name': 'Shaka', 'civ': 'Zulu', 'isMajor': True}
{'leaderId': None, 'name': 'Poundmaker', 'civ': 'Cree', 'isMajor': True}
{'leaderId': None, 'name': 'Lautaro', 'civ': 'Mapuche', 'isMajor': True}
{'leaderId': None, 'name': 'Vilnius', 'civ': 'Vilnius', 'isMajor': False}
{'leaderId': None, 'name': 'Granada', 'civ': 'Granada', 'isMajor': False}
{'leaderId': None, 'name': 'Valletta', 'civ': 'Valletta', 'isMajor': False}
{'leaderId': None, 'name': 'Buenos Aires', 'civ': 'Buenos Aires', 'isMajor': False}
{'leaderId': None, 'name': 'Carthage', 'civ': 'Carthage', 'isMajor': False}


In [16]:
for row in result:
#    print(row)
    cur.execute('INSERT INTO leader VALUES(:leaderId, :name, :civ, :isMajor)', row)

In [17]:
cnx.commit()

# Populate gameLeader table

In [18]:
knownLeaders = getKnownLeaders(cnx)
print(knownLeaders)
humanLeaderName = gameData['leader']
print(humanLeaderName)
print(gameId)

    leaderId                 name            civ
0          7            Amanitore          Nubia
1         35              Antioch        Antioch
2         14               Armagh         Armagh
3         28             Auckland       Auckland
4          4              Babylon        Babylon
5          3        Bandar Brunei  Bandar Brunei
6         13             Brussels       Brussels
7         47         Buenos Aires   Buenos Aires
8         48             Carthage       Carthage
9         11  Catherine de Medici         France
10        22            Cleopatra          Egypt
11        26                Cyrus         Persia
12        12               Geneva         Geneva
13        21         Genghis Khan       Mongolia
14         1            Gilgamesh        Sumeria
15        20              Gitarja      Indonesia
16        45              Granada        Granada
17        24      Harald Hardrada         Norway
18        36              Hattusa        Hattusa
19         5        

In [19]:
result = []
for leader in gameData['players']:
    name = leader['playerName']
    civ = leader['playerCiv']
    leaderId = getLeaderId(name, civ, knownLeaders)
    if name == humanLeaderName:
        newGameLeader = {
            'gameId': gameId,
            'leaderId': leaderId,
            'isHuman': True,
        }
        result.append(newGameLeader)
    else:
        newGameLeader = {
            'gameId': gameId,
            'leaderId': leaderId,
            'isHuman': False,
        }
        result.append(newGameLeader)
    
for row in result:
    print(row)

{'gameId': 4, 'leaderId': 10, 'isHuman': True}
{'gameId': 4, 'leaderId': 38, 'isHuman': False}
{'gameId': 4, 'leaderId': 39, 'isHuman': False}
{'gameId': 4, 'leaderId': 40, 'isHuman': False}
{'gameId': 4, 'leaderId': 41, 'isHuman': False}
{'gameId': 4, 'leaderId': 21, 'isHuman': False}
{'gameId': 4, 'leaderId': 42, 'isHuman': False}
{'gameId': 4, 'leaderId': 43, 'isHuman': False}
{'gameId': 4, 'leaderId': 13, 'isHuman': False}
{'gameId': 4, 'leaderId': 29, 'isHuman': False}
{'gameId': 4, 'leaderId': 35, 'isHuman': False}
{'gameId': 4, 'leaderId': 33, 'isHuman': False}
{'gameId': 4, 'leaderId': 15, 'isHuman': False}
{'gameId': 4, 'leaderId': 44, 'isHuman': False}
{'gameId': 4, 'leaderId': 45, 'isHuman': False}
{'gameId': 4, 'leaderId': 46, 'isHuman': False}
{'gameId': 4, 'leaderId': 30, 'isHuman': False}
{'gameId': 4, 'leaderId': 47, 'isHuman': False}
{'gameId': 4, 'leaderId': 32, 'isHuman': False}
{'gameId': 4, 'leaderId': 48, 'isHuman': False}


In [20]:
for row in result:
#    print(row)
    cur.execute('INSERT INTO gameLeader VALUES(:gameId, :leaderId, :isHuman)', row)

In [21]:
cnx.commit()

# Populate citySettled and gameCity Tables

In [22]:
citySettledLog = rawData['cityOnSettledLog']
#print(citySettledLog)
for city in citySettledLog:
    ownerName = city['ownerName']
    name = city['cityName']
    leaderId = int(knownLeaders[(knownLeaders.name == ownerName)].iloc[0].leaderId)

    sqlInsert = "INSERT INTO citySettled VALUES(:cityId, :name, :settledBy, :onTurn)"
    parameters = {'cityId': None, 'name': name, 'settledBy':leaderId, 'onTurn':city['turn']}
    cur.execute(sqlInsert, parameters)
    cityId = cur.lastrowid
    print(cityId, ownerName, name)
    
    sqlInsert = "INSERT INTO gameCity VALUES(:gameId, :cityId)"
    parameters = {'gameId':gameId, 'cityId':cityId}
    cur.execute(sqlInsert, parameters)

50 Peter St. Petersburg
51 Teddy Roosevelt Washington
52 Mvemba a Nzinga Mbanza Kongo
53 Shaka Ulundi
54 Poundmaker Mikisiw-Wacîhk
55 Brussels Brussels
56 Nan Madol Nan Madol
57 Antioch Antioch
58 Muscat Muscat
59 Stockholm Stockholm
60 Vilnius Vilnius
61 Granada Granada
62 Valletta Valletta
63 Jerusalem Jerusalem
64 Buenos Aires Buenos Aires
65 Zanzibar Zanzibar
66 Carthage Carthage
67 Pericles Athens
68 Genghis Khan Qaraqorum
69 Lautaro Ngulu Mapu


In [ ]:
#cnx.rollback()

In [23]:
cnx.commit()

## Populate cityPlotsSettled

In [24]:
sqlSelect = 'SELECT * FROM gameCityLeaderSettledView WHERE gameId = :gameId'
gameCLS = pd.read_sql_query(sqlSelect, cnx, params={'gameId':gameId})
print(gameCLS)

    gameId  cityId        cityName       leaderName  leaderId     leaderCiv  \
0        4      50  St. Petersburg            Peter        10        Russia   
1        4      51      Washington  Teddy Roosevelt        38       America   
2        4      52    Mbanza Kongo  Mvemba a Nzinga        40         Kongo   
3        4      53          Ulundi            Shaka        41          Zulu   
4        4      54  Mikisiw-Wacîhk       Poundmaker        42          Cree   
5        4      55        Brussels         Brussels        13      Brussels   
6        4      56       Nan Madol        Nan Madol        29     Nan Madol   
7        4      57         Antioch          Antioch        35       Antioch   
8        4      58          Muscat           Muscat        33        Muscat   
9        4      59       Stockholm        Stockholm        15     Stockholm   
10       4      60         Vilnius          Vilnius        44       Vilnius   
11       4      61         Granada          Granada 

In [ ]:
# Testing, can safely ignore this!
#ownerCityId = int(gameCLS[gameCLS.cityName == 'Amsterdam'].iloc[0].cityId)
#print(ownerCityId)
#print(type(ownerCityId))

In [25]:
# building up the cityPlotsSettled data for entry...
result = []
for city in rawData['cityOnSettledLog']:
    for plot in city['plots']:
        try:
            ownerCityId = int(gameCLS[gameCLS.cityName == plot['ownerCity']].iloc[0].cityId)
        except IndexError:
            ownerCityId = None
        plotInfo = {
            'plotId': None,
            'ownerCityId': ownerCityId,
            'ring': plot['r'],
            'terrain': plot['terrain'],
            'feature': plot['feature'],
            'resource': plot['resource'],
            'resourceCount': plot['resourceCount'],
            'resourceType': plot['resourceType'],
            'workers': plot['workers'],
            'district': plot['district'],
            'hasRiver': plot['hasRiver'],
            'isWater': plot['isWater'],
            'isLake': plot['isLake'],
            'isCity': plot['isCity'],
        }
        result.append(plotInfo)

print(result[0])
print(result[1])
print(result[2])

{'plotId': None, 'ownerCityId': 50, 'ring': 0, 'terrain': 'Plains (Hills)', 'feature': 'None', 'resource': 'None', 'resourceCount': 0, 'resourceType': 'None', 'workers': 1, 'district': 'City Center', 'hasRiver': True, 'isWater': False, 'isLake': False, 'isCity': True}
{'plotId': None, 'ownerCityId': 50, 'ring': 1, 'terrain': 'Plains', 'feature': 'None', 'resource': 'Cotton', 'resourceCount': 1, 'resourceType': 'Luxury', 'workers': 0, 'district': 'None', 'hasRiver': True, 'isWater': False, 'isLake': False, 'isCity': False}
{'plotId': None, 'ownerCityId': 50, 'ring': 1, 'terrain': 'Plains', 'feature': 'None', 'resource': 'Wheat', 'resourceCount': 1, 'resourceType': 'Bonus', 'workers': 0, 'district': 'None', 'hasRiver': True, 'isWater': False, 'isLake': False, 'isCity': False}


In [26]:
for row in result:
    cur.execute('INSERT INTO cityPlotsSettled VALUES(:plotId, :ownerCityId, :ring, :terrain, :feature, :resource, :resourceCount, :resourceType, :workers, :district, :hasRiver, :isWater, :isLake, :isCity )',
                row)

In [ ]:
#cnx.rollback()

In [27]:
cnx.commit()

## Populate cityPerTurnLog

In [28]:
def getCityId(cityName):
    try:
        # The iloc is neccessary as we can't assume there is only one return value!?
        # also have to make sure it is int as numpy.int64 doesn't go into sqlite so well?
        return int(gameCLS[gameCLS.cityName == cityName].iloc[0].cityId)
    except IndexError:
        return None
    
def getLeaderId(ownerName, ownerCiv):
    try:
        # The iloc is necessary as we can't assume there is only one return value!?
        # also have to make sure it is int as numpy.int64 doesn't go into sqlite so well?
        return int(gameCLS[gameCLS.leaderName  == ownerName].iloc[0].leaderId)
    except IndexError:
        return None

In [29]:
# building up the cityPerTurnLog data for entry...
result = []
for logEntry in rawData['cityPerTurnLog']:
    turnInfo = {
        'cityId': getCityId(logEntry['cityName']),
        'turn': int(logEntry['turn']),
        'ownerId': getLeaderId(logEntry['ownerName'], 'Blah'),
        'foodPerTurn': logEntry['foodPerTurn'],
        'foodToolTip': logEntry['foodToolTip'],
        'productionPerTurn': logEntry['productionPerTurn'],
        'productionToolTip': logEntry['productionToolTip'],
        'goldPerTurn': logEntry['goldPerTurn'],
        'goldToolTip': logEntry['goldToolTip'],
        'sciencePerTurn': logEntry['sciencePerTurn'],
        'scienceToolTip': logEntry['scienceToolTip'],
        'culturePerTurn': logEntry['culturePerTurn'],
        'cultureToolTip': logEntry['cultureToolTip'],
        'faithPerTurn': logEntry['faithPerTurn'],
        'faithToolTip': logEntry['faithToolTip'],
        'population': logEntry['population'],
        'housing': logEntry['housing'],
        'amenities': logEntry['amenities'],
        'amenitiesNeeded': logEntry['amenitiesNeeded'],
        'happiness': logEntry['happiness'],
    }
    result.append(turnInfo)

print(result[0])
print(result[1])
print(result[-1])

{'cityId': 50, 'turn': 1, 'ownerId': 10, 'foodPerTurn': '3.00', 'foodToolTip': '+3 from Worked Tiles', 'productionPerTurn': '6.30', 'productionToolTip': '+4 from Worked Tiles[NEWLINE]+2 from Buildings[NEWLINE]   +2 from Palace[NEWLINE]+5pct (+0.2) from Amenities', 'goldPerTurn': '7.35', 'goldToolTip': '+2 from Worked Tiles[NEWLINE]+5 from Buildings[NEWLINE]   +5 from Palace[NEWLINE]+5pct (+0.3) from Amenities', 'sciencePerTurn': '2.63', 'scienceToolTip': '+2 from Buildings[NEWLINE]   +2 from Palace[NEWLINE]+0.5 from Population[NEWLINE]+5pct (+0.1) from Amenities', 'culturePerTurn': '1.36', 'cultureToolTip': '+1 from Buildings[NEWLINE]   +1 from Palace[NEWLINE]+0.2 from Population[NEWLINE]+5pct (+0) from Amenities', 'faithPerTurn': '0.00', 'faithToolTip': '+5pct (+0) from Amenities', 'population': 1, 'housing': 6, 'amenities': 1, 'amenitiesNeeded': 0, 'happiness': 'Happy'}
{'cityId': 51, 'turn': 1, 'ownerId': 38, 'foodPerTurn': '4.00', 'foodToolTip': '+4 from Worked Tiles', 'productionP

In [30]:
for row in result:
    cur.execute('INSERT INTO cityPerTurnLog VALUES(:cityId, :turn, :ownerId, :foodPerTurn, :foodToolTip, :productionPerTurn, :productionToolTip, :goldPerTurn, :goldToolTip, :sciencePerTurn, :scienceToolTip, :culturePerTurn, :cultureToolTip, :faithPerTurn, :faithToolTip, :population, :housing, :amenities, :amenitiesNeeded, :happiness)',
                row)

In [ ]:
cnx.rollback()

In [31]:
cnx.commit()

In [32]:
cnx.close()